In [ ]:
##Executed after LSTM_long_final.ipynb, or CNN variant of code
##Since the predictions in long-format have yields that are supposed to represent
##annual yields repeating 19 times, they are to be averaged and have their
##evaluation metrics done again.

##import the libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_array
from sklearn.metrics import mean_squared_error

!pip install hydroeval
import hydroeval as he

In [ ]:
##Define MAPE function
def mean_absolute_percentage_error(y_t, y_p): 
    
  return np.mean(np.abs((y_t - y_p) / y_t)) * 100

In [ ]:
##Get directory containing 
direct = "/content/drive/MyDrive/Canola_predict"
##Set name of crop being processed
crop = "Canola"
lof = sorted(os.listdir(direct))

##Set up empty array for dataframe to be written to .csv file
car = []
r2_car = []
mape_car = []
nse_car = []
rmse_car = []

all_car = []
years = []
all_actual = []
all_pred = []

##For each .csv file containing predicted yields
for f in lof:

  df = pd.read_csv(direct + "/" + f)

  ##Get the starting year of the caruid
  start = int((df["Year"][0])[0:4])
  ##Get length of year range
  year_len = int(len(df)/19)

  ##Get caruid
  car.append(int(f[7:-4]))

  ##Empty array for caruid's actual and predicted yield
  local_actual = []
  local_pred = []

  ##Average each yield and actual yield of each year
  for i in range(0, year_len):
    actual_yld = np.mean(df["Yield"][(0 + i * 19):(18 + i * 19)])
    pred_yld = np.mean(df["Predict"][(0 + i * 19):(18 + i * 19)])

    local_actual.append(actual_yld)
    all_actual.append(actual_yld)
    local_pred.append(pred_yld)
    all_pred.append(pred_yld)

    all_car.append(int(f[7:-4]))
    years.append(start + i)
  
  ##Do all the evaluation metrics of the "annualized" prediction values
  correlation_matrix = np.corrcoef(local_actual, local_pred)
  correlation_xy = correlation_matrix[0,1]
  r2 = correlation_xy**2

  mape = mean_absolute_percentage_error(np.array(local_actual), np.array(local_pred))
  nse = he.evaluator(he.nse, local_pred, local_actual)

  rmse = mean_squared_error(local_actual, local_pred, squared=False)

  r2_car.append(r2)
  mape_car.append(mape)
  nse_car.append(nse[0])
  rmse_car.append(rmse)

##First write "annualized" yield prediction to .csv file that contains all caruid
forecast_df = pd.DataFrame(data={"Year": years, "CARUID": all_car, "Predicted": all_pred,
                   "Actual": all_actual})
forecast_df.to_csv("/content/drive/MyDrive/" + crop + "_annualized_YieldForecast/" + crop + "_annualized_forecast.csv", index=False)

##Write to .csv file which contains evaluation metric for "annualized" prediction
performance_df = pd.DataFrame(data={"CARUID": car, "R_squared": r2_car,
                   "MAPE": mape_car, "MEI": nse_car, "RMSE": rmse_car})
performance_df.to_csv("/content/drive/MyDrive/" + crop + "_annualized_YieldForecast/" + crop + "_annual_performance.csv", index=False)

In [ ]:
actual_yld

29.5